In [2]:
import pandas as pd
import numpy as np
import re

from astropy.coordinates import SkyCoord
import astropy.units as u

# ======================================
# INPUT / OUTPUT
# ======================================
input_csv        = "../paper1/csv/combined_sources_with_alpha.csv"
output_mrt       = "../paper1/csv/combined_sources_with_alpha_mrt.txt"
output_data_only = "../paper1/csv/combined_sources_with_alpha_data_only.txt"

# ======================================
# IAU-COMPLIANT SOURCE NAME (STRICT TRUNCATION)
# Format: VCPMC JHHMMSS.s+DDMMSS
# ======================================
def make_vcpmc_name(ra_deg, dec_deg, prefix="VCPMC"):
    coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg, frame="icrs")

    # RA: truncate to 0.1 s
    ra_h = coord.ra.hms.h
    ra_m = coord.ra.hms.m
    ra_s = coord.ra.hms.s
    ra_s_trunc = np.floor(ra_s * 10.0) / 10.0
    ra_str = f"{int(ra_h):02d}{int(ra_m):02d}{ra_s_trunc:04.1f}"  # HHMMSS.s

    # Dec: truncate to integer arcsec
    dec_sign = "+" if coord.dec.deg >= 0 else "-"
    dec_d = abs(coord.dec.dms.d)
    dec_m = coord.dec.dms.m
    dec_s = coord.dec.dms.s
    dec_s_trunc = int(np.floor(dec_s))
    dec_str = f"{dec_sign}{int(dec_d):02d}{int(dec_m):02d}{dec_s_trunc:02d}"  # +DDMMSS

    return f"{prefix} J{ra_str}{dec_str}"


# ======================================
# LOAD
# ======================================
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# Clean Source_ID if it exists
if "Source_ID" in df.columns:
    df["Source_ID"] = df["Source_ID"].astype(str).str.strip()

# ======================================
# FORCE-UPDATE Source_ID TO NEW FORMAT
# - safest: regenerate from RA/DEC to guarantee truncation
# - also updates prefix to VCPMC
# ======================================
required_pos_cols = {"RA", "DEC"}
if not required_pos_cols.issubset(df.columns):
    raise KeyError(f"CSV must contain columns {required_pos_cols}. Found: {df.columns.tolist()}")

df["Source_ID"] = df.apply(lambda r: make_vcpmc_name(float(r["RA"]), float(r["DEC"]), prefix="VCPMC"), axis=1)

# ======================================
# STANDARDIZE COLUMN NAMES
# ======================================
rename_dict = {}
for col in df.columns:
    if "_spw" in col:
        rename_dict[col] = col.replace("_spw", "_SPW")
df = df.rename(columns=rename_dict)

# ======================================
# MAP SPWs from original to 1–9
# ======================================
spw_map = {15: 1, 16: 2, 17: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 8, 8: 9}

remap_dict = {}
for col in df.columns:
    m = re.search(r"_SPW(\d+)", col)
    if m:
        old_spw = int(m.group(1))
        if old_spw in spw_map:
            new_spw = spw_map[old_spw]
            remap_dict[col] = re.sub(r"_SPW\d+", f"_SPW{new_spw}", col)

df = df.rename(columns=remap_dict)
print("✅ Detected columns after remapping SPWs:", df.columns.tolist())

# ======================================
# DEFINE COLUMN GROUPS
# ======================================
ra_dec_cols  = ["RA", "DEC", "E_RA", "E_DEC"]
alpha_cols   = ["alpha", "E_alpha"]
isl_rms_cols = [c for c in df.columns if c.startswith("RMS_SPW")]
flux_cols    = [c for c in df.columns
                if c.startswith("Total_S_SPW")
                or c.startswith("E_Total_S_SPW")
                or c.startswith("Peak_S_SPW")
                or c.startswith("E_Peak_S_SPW")]

columns = ["Source_ID"] + ra_dec_cols + alpha_cols + flux_cols + isl_rms_cols

# Keep only columns that exist (prevents KeyError if a column is missing)
columns = [c for c in columns if c in df.columns]

# ======================================
# CONVERT RMS from mJy -> µJy
# ======================================
for c in isl_rms_cols:
    if c in df.columns:
        df[c] = df[c] * 1000.0

# ======================================
# HEADER ROWS
# ======================================
labels = columns

# Units
units = []
for col in columns:
    if col in ["RA", "DEC", "E_RA", "E_DEC"]:
        units.append("deg")
    elif col in ["alpha", "E_alpha"]:
        units.append("---")
    elif (col.startswith("Total_S_SPW") or
          col.startswith("E_Total_S_SPW") or
          col.startswith("Peak_S_SPW") or
          col.startswith("E_Peak_S_SPW")):
        units.append("mJy")
    elif col.startswith("RMS_SPW"):
        units.append("uJy")
    elif col == "Source_ID":
        units.append("---")
    else:
        units.append("---")

# Explanations
explanations = []
for col in columns:
    if col == "Source_ID":
        explanations.append("IAU-compliant source name (VCPMC JHHMMSS.s+DDMMSS; coordinates truncated, not rounded)")
    elif col == "RA":
        explanations.append("Right ascension (J2000) in degrees")
    elif col == "DEC":
        explanations.append("Declination (J2000) in degrees")
    elif col == "E_RA":
        explanations.append("Error in RA [deg]")
    elif col == "E_DEC":
        explanations.append("Error in DEC [deg]")
    elif col == "alpha":
        explanations.append("Spectral index")
    elif col == "E_alpha":
        explanations.append("Error in spectral index")
    elif col.startswith("Total_S_SPW"):
        explanations.append("Integrated flux density in this SPW")
    elif col.startswith("E_Total_S_SPW"):
        explanations.append("Error on integrated flux density in this SPW")
    elif col.startswith("Peak_S_SPW"):
        explanations.append("Peak flux density in this SPW")
    elif col.startswith("E_Peak_S_SPW"):
        explanations.append("Error on peak flux density in this SPW")
    elif col.startswith("RMS_SPW"):
        explanations.append("Island RMS noise in this SPW")
    else:
        explanations.append("")

# ======================================
# UPDATE FORMAT CODES FOR NEW Source_ID
# New ID length:
#   "VCPMC J" (6) + "HHMMSS.s" (8) + "+DDMMSS" (7) = 21 characters total
# Example: "VCPMC J034012.7+314512" -> 21
# We'll set A21 (or A22 for safety). Use A21 here.
# ======================================
formats = []
for col in columns:
    if col == "Source_ID":
        formats.append("A21")
    elif col in ["RA", "DEC", "E_RA", "E_DEC"]:
        formats.append("F10.5")
    elif col in ["alpha", "E_alpha"]:
        formats.append("F6.2")
    else:
        formats.append("F8.2")

# Python fixed-width formats (for alignment in the text file)
py_formats = []
for col in columns:
    if col == "Source_ID":
        py_formats.append("s21")     # match A21
    elif col in ["RA", "DEC", "E_RA", "E_DEC"]:
        py_formats.append("f10.5")
    elif col in ["alpha", "E_alpha"]:
        py_formats.append("f6.2")
    else:
        py_formats.append("f8.2")

# ======================================
# HELPER: FIXED-WIDTH PRINTING
# ======================================
def format_fixed(x, fmt):
    """
    fmt: 's21', 'i5', 'f8.2', etc. (Python-style helper, NOT the AAS code)
    """
    if pd.isna(x):
        spec = fmt[0]
        if spec in ["f", "i"]:
            width = int(fmt[1:].split(".")[0]) if "." in fmt[1:] else int(fmt[1:])
            return "".rjust(width)
        # strings: return blanks of width so columns stay aligned
        if spec == "s":
            width = int(fmt[1:])
            return "".rjust(width)
        return ""

    spec = fmt[0]
    if spec == "s":
        width = int(fmt[1:])
        return f"{str(x):>{width}s}"
    if spec == "i":
        width = int(fmt[1:])
        return f"{int(x):>{width}d}"
    if spec == "f":
        width_precision = fmt[1:].split(".")
        width = int(width_precision[0])
        precision = int(width_precision[1])
        return f"{float(x):>{width}.{precision}f}"
    return str(x)

# ======================================
# FORMAT DATA ROWS
# ======================================
formatted_rows = []
for _, r in df.iterrows():
    formatted_row = []
    for col, pyfmt in zip(columns, py_formats):
        formatted_row.append(format_fixed(r[col], pyfmt))
    formatted_rows.append(" ".join(formatted_row))

# ======================================
# WRITE MRT (header + data)
# ======================================
with open(output_mrt, "w") as f:
    f.write(" ".join(labels) + "\n")
    f.write(" ".join(units) + "\n")
    f.write(" ".join(explanations) + "\n")
    f.write(" ".join(formats) + "\n")
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved MRT file with full header to: {output_mrt}")

# ======================================
# WRITE DATA-ONLY
# ======================================
with open(output_data_only, "w") as f:
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved data-only file (formatted & no header) to: {output_data_only}")

# ======================================
# PRINT THE FORMAT CODE STRING (AAS box)
# ======================================
print("\n👉 Copy/paste this format code into the AAS 'format code' box:\n")
print(" ".join(formats))


✅ Detected columns after remapping SPWs: ['Source_ID', 'RA', 'DEC', 'E_RA', 'E_DEC', 'Total_S_SPW1', 'E_Total_S_SPW1', 'Peak_S_SPW1', 'E_Peak_S_SPW1', 'RMS_SPW1', 'Total_S_SPW2', 'E_Total_S_SPW2', 'Peak_S_SPW2', 'E_Peak_S_SPW2', 'RMS_SPW2', 'Total_S_SPW3', 'E_Total_S_SPW3', 'Peak_S_SPW3', 'E_Peak_S_SPW3', 'RMS_SPW3', 'Total_S_SPW4', 'E_Total_S_SPW4', 'Peak_S_SPW4', 'E_Peak_S_SPW4', 'RMS_SPW4', 'Total_S_SPW5', 'E_Total_S_SPW5', 'Peak_S_SPW5', 'E_Peak_S_SPW5', 'RMS_SPW5', 'Total_S_SPW6', 'E_Total_S_SPW6', 'Peak_S_SPW6', 'E_Peak_S_SPW6', 'RMS_SPW6', 'Total_S_SPW7', 'E_Total_S_SPW7', 'Peak_S_SPW7', 'E_Peak_S_SPW7', 'RMS_SPW7', 'Total_S_SPW8', 'E_Total_S_SPW8', 'Peak_S_SPW8', 'E_Peak_S_SPW8', 'RMS_SPW8', 'Total_S_SPW9', 'E_Total_S_SPW9', 'Peak_S_SPW9', 'E_Peak_S_SPW9', 'RMS_SPW9', 'alpha', 'E_alpha']
✅ Saved MRT file with full header to: ../paper1/csv/combined_sources_with_alpha_mrt.txt
✅ Saved data-only file (formatted & no header) to: ../paper1/csv/combined_sources_with_alpha_data_only.tx

# Latex

In [3]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

# === File paths ===
spw15_path = '../paper1/csv/spw15.csv'
combined_sources_path = '../paper1/csv/combined_sources_with_alpha.csv'  # or _with_existing_alpha.csv if you prefer

# === Load CSV files ===
df_spw15 = pd.read_csv(spw15_path)
df_combined = pd.read_csv(combined_sources_path)

# === SkyCoord for RA/DEC matching ===
coords_spw15 = SkyCoord(ra=df_spw15['RA'].values * u.deg,
                        dec=df_spw15['DEC'].values * u.deg)
coords_combined = SkyCoord(ra=df_combined['RA'].values * u.deg,
                           dec=df_combined['DEC'].values * u.deg)

# === Nearest neighbor match within 40 arcsec ===
idx, d2d, _ = coords_spw15.match_to_catalog_sky(coords_combined)
tolerance = 40 * u.arcsec
matched_mask = d2d < tolerance

# === Matched DataFrame ===
df_matched = df_spw15[matched_mask].copy()

# Attach Source_ID and (optionally) RA/DEC, E_RA, E_DEC from combined catalog
df_matched['Source_ID'] = df_combined.iloc[idx[matched_mask]]['Source_ID'].values
df_matched['RA']        = df_combined.iloc[idx[matched_mask]]['RA'].values
df_matched['DEC']       = df_combined.iloc[idx[matched_mask]]['DEC'].values

if 'E_RA' in df_combined.columns and 'E_DEC' in df_combined.columns:
    df_matched['E_RA']  = df_combined.iloc[idx[matched_mask]]['E_RA'].values
    df_matched['E_DEC'] = df_combined.iloc[idx[matched_mask]]['E_DEC'].values

# === Convert RMS from mJy to µJy ===
# Isl_rms in spw15 is in mJy; convert to µJy for the table
if 'Isl_rms' in df_matched.columns:
    df_matched['Isl_rms'] = df_matched['Isl_rms'] * 1000.0

# === Define columns & LaTeX header (SPW15 → SPW1 in naming) ===
column_order = [
    'Source_ID',
    'RA', 'E_RA',
    'DEC', 'E_DEC',
    'Total_flux', 'E_Total_flux',
    'Peak_flux', 'E_Peak_flux',
    'Isl_rms'
]

# We will *display* them with these LaTeX labels:
latex_header = [
    'Source\\_ID',
    'RA', 'E\\_RA',
    'DEC', 'E\\_DEC',
    'Total\\_S\\_SPW1', 'E\\_Total\\_S\\_SPW1',
    'Peak\\_S\\_SPW1', 'E\\_Peak\\_S\\_SPW1',
    'RMS\\_SPW1'
]

latex_units = [
    '---',
    'deg', 'deg',
    'deg', 'deg',
    'mJy', 'mJy',
    'mJy', 'mJy',
    '$\\mu$Jy'
]

# === Keep available columns only and take first 10 ===
available_cols = [c for c in column_order if c in df_matched.columns]
df_final = df_matched[available_cols].head(10).copy()

# Make sure header & units are consistent with available columns
col_to_label = dict(zip(column_order, latex_header))
col_to_unit  = dict(zip(column_order, latex_units))

labels = [col_to_label[c] for c in available_cols]
units  = [col_to_unit[c]  for c in available_cols]

# === Generate LaTeX table in the requested style ===
colspec = 'l' * len(available_cols)  # e.g., llllllllll

latex_table = r"""\begin{table*}[ht!]
\scriptsize
\setlength{\tabcolsep}{2pt}
\renewcommand{\arraystretch}{1.1}
\centering
\caption{Example rows from the catalog showing RA, Dec, flux densities, and associated uncertainties for sources in SPW1. The full catalog includes all 9 SPWs and is available online in machine-readable format.}
\label{tab:spw15-matched}
\begin{tabular}{%s}
\toprule
%s \\
%s \\
\midrule
""" % (
    colspec,
    ' & '.join(labels),
    ' & '.join(units)
)

# === Add table rows ===
for _, row in df_final.iterrows():
    row_entries = []
    for col in available_cols:
        val = row[col]
        if isinstance(val, (float, int)) and not pd.isna(val):
            # RA/DEC and their errors: 5 decimals
            if col in ['RA', 'DEC', 'E_RA', 'E_DEC']:
                row_entries.append(f"{val:.5f}")
            else:
                # fluxes and RMS: 2 decimals
                row_entries.append(f"{val:.2f}")
        else:
            row_entries.append(str(val))
    latex_table += ' & '.join(row_entries) + r" \\" + "\n"

latex_table += r"""\bottomrule
\end{tabular}
\end{table*}
"""

print(latex_table)


\begin{table*}[ht!]
\scriptsize
\setlength{\tabcolsep}{2pt}
\renewcommand{\arraystretch}{1.1}
\centering
\caption{Example rows from the catalog showing RA, Dec, flux densities, and associated uncertainties for sources in SPW1. The full catalog includes all 9 SPWs and is available online in machine-readable format.}
\label{tab:spw15-matched}
\begin{tabular}{llllllllll}
\toprule
Source\_ID & RA & E\_RA & DEC & E\_DEC & Total\_S\_SPW1 & E\_Total\_S\_SPW1 & Peak\_S\_SPW1 & E\_Peak\_S\_SPW1 & RMS\_SPW1 \\
--- & deg & deg & deg & deg & mJy & mJy & mJy & mJy & $\mu$Jy \\
\midrule
VCPMC J033003.6+303255 & 52.51526 & 0.00158 & 30.54888 & 0.00195 & 149.17 & 28.71 & 54.13 & 7.93 & 7.42 \\
VCPMC J033002.9+303533 & 52.51231 & 0.00100 & 30.59268 & 0.00167 & 40.98 & 12.38 & 39.33 & 6.87 & 6.90 \\
VCPMC J032959.1+303622 & 52.49627 & 0.00327 & 30.60620 & 0.00122 & 19.32 & 10.67 & 21.85 & 6.39 & 6.90 \\
VCPMC J032940.2+312455 & 52.41754 & 0.00023 & 31.41532 & 0.00021 & 316.33 & 28.67 & 270.64 & 14.90 & 

# Finding tolerance suitable for matches

In [4]:
import pandas as pd

# ======================================
# Input file
# ======================================
input_csv = "../paper1/csv/combined_sources_with_alpha.csv"

# ======================================
# Load data
# ======================================
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# ======================================
# Ensure numeric values
# ======================================
for col in ["E_RA", "E_DEC"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ======================================
# Conversion factor
# ======================================
DEG_TO_ARCSEC = 3600.0

# ======================================
# Statistics (degrees)
# ======================================
era_mean_deg    = df["E_RA"].mean()
era_median_deg  = df["E_RA"].median()
era_max_deg     = df["E_RA"].max()

edec_mean_deg   = df["E_DEC"].mean()
edec_median_deg = df["E_DEC"].median()
edec_max_deg    = df["E_DEC"].max()

# ======================================
# Statistics (arcseconds)
# ======================================
era_mean_arcsec    = era_mean_deg    * DEG_TO_ARCSEC
era_median_arcsec  = era_median_deg  * DEG_TO_ARCSEC
era_max_arcsec     = era_max_deg     * DEG_TO_ARCSEC

edec_mean_arcsec   = edec_mean_deg   * DEG_TO_ARCSEC
edec_median_arcsec = edec_median_deg * DEG_TO_ARCSEC
edec_max_arcsec    = edec_max_deg    * DEG_TO_ARCSEC

# ======================================
# Print results
# ======================================
print("=== Positional Uncertainty Statistics ===")

print("\nUnits: degrees")
print(f"E_RA  : mean = {era_mean_deg:.6e}, median = {era_median_deg:.6e}, max = {era_max_deg:.6e}")
print(f"E_DEC : mean = {edec_mean_deg:.6e}, median = {edec_median_deg:.6e}, max = {edec_max_deg:.6e}")

print("\nUnits: arcseconds")
print(f"E_RA  : mean = {era_mean_arcsec:.3f}, median = {era_median_arcsec:.3f}, max = {era_max_arcsec:.3f} arcsec")
print(f"E_DEC : mean = {edec_mean_arcsec:.3f}, median = {edec_median_arcsec:.3f}, max = {edec_max_arcsec:.3f} arcsec")


=== Positional Uncertainty Statistics ===

Units: degrees
E_RA  : mean = 9.675500e-04, median = 8.109197e-04, max = 4.625950e-03
E_DEC : mean = 9.081782e-04, median = 8.271066e-04, max = 4.258153e-03

Units: arcseconds
E_RA  : mean = 3.483, median = 2.919, max = 16.653 arcsec
E_DEC : mean = 3.269, median = 2.978, max = 15.329 arcsec
